In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Use the requests library to download the content of the webpage

In [2]:
URL = 'https://barttorvik.com/trank.php#'
page = requests.get(URL)

Turn the BeautifulSoup and html.parser to convert the page html data to a BeautifulSoup object

In [3]:
raw_html = BeautifulSoup(page.content, 'html.parser')

Take a look at the text in the first h1 tag in the page

In [4]:
test = raw_html.find('h1')
print(test.text)


2021 Customizable T-Rank and Tempo-Free Stats 


The columns name are under thead > tr > th. here is how we get them: (They all have a style in common so we use that to filter out the titles)

In [5]:
table_head = raw_html.find('thead')
cols = table_head.find('tr', style="font-size:10px")
colHead = cols.find_all('th')
c = 0
column_names = []
for i in colHead:
    column_names.append(i.text)
column_names

['Rk',
 'Team',
 'Conf',
 'G',
 'Rec',
 'AdjOE',
 'AdjDE',
 'Barthag',
 'EFG%',
 'EFGD%',
 'TOR',
 'TORD',
 'ORB',
 'DRB',
 'FTR',
 'FTRD',
 '2P%',
 '2P%D',
 '3P%',
 '3P%D',
 'Adj T.',
 'WAB']

The table data is under tbody > tr > td

In [6]:
table_Body = raw_html.find("tbody")
body_data = table_Body.find_all('tr')
all_data=[]
for rows in body_data:
    r = 0
    signleRowData = rows.find_all('td', recursive=False)
    if len(signleRowData) > 0:
        team_d ={}
        for col in signleRowData:
            #There are some incosistancies on how table columns are set up
            #We use a if statment to get the right data
            if r > 3:
                data = col.find('br').previousSibling
                #Each row (aka each team) is added to a dictionery {column_name: column_value}
                team_d.update({column_names[r]: str(data.string)})
            else:
                team_d.update({column_names[r]: col.text})
            r += 1 
        #The dictioneries are added to an array
        all_data.append(team_d)
#The array of dictioneries are turned into a pandas dataframe
df2021 = pd.DataFrame(all_data, columns=column_names)
df2021.head(6)

,Rk,Team,Conf,G,Rec,AdjOE,AdjDE,Barthag,EFG%,EFGD%,...,ORB,DRB,FTR,FTRD,2P%,2P%D,3P%,3P%D,Adj T.,WAB
0,1,Gonzaga,WCC,24,24-0,124.8,89.1,.9797,61.1,47.5,...,30.8,23.7,37,26.8,64.5,46.5,36,33,74.9,7.6
1,2,Houston,Amer,22,20-3,117.9,88.5,.9646,50.8,42.4,...,39.5,27.9,31.4,43.6,49.2,42.6,35.3,28.1,65.9,2.9
2,3,Baylor (H) 32 Oklahoma St.,B12,20,19-1,121.7,91.9,.9620,57.8,48.2,...,37,31.1,26.8,31.4,54.4,47.9,42.1,32.6,69.2,6.1
3,4,Michigan (H) 60 Michigan St.,B10,20,18-2,118.2,89.6,.9602,55.6,44.3,...,29.9,24.7,29.9,24,54.3,41.6,38.8,33.2,67.8,7.3
4,5,Iowa (H) 82 Nebraska,B10,25,18-7,124.0,96.3,.9487,55,48.3,...,31.7,29.7,33.4,23.4,51.6,45.6,40.1,34.9,70,4.6
5,6,Illinois,B10,25,19-6,116.0,90.4,.9461,55.9,46.3,...,32.6,22.1,38.4,30.4,55.2,44.7,38.3,33.1,71,6.5


In [7]:
df2021.shape

(347, 22)

In [8]:
df2021.to_csv("df2021.csv")